## Necessary Module Importation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import dotenv_values
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import gdown
import logging
import pyodbc
import datetime as dt
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

## Data Loading and Storage

In [6]:
Holidays_Data = pd.read_csv('C:\\Users\\USER\\Downloads\\holiday_data.csv')
Holidays_Data.head()

,Unnamed: 0,date,type,locale,locale_name,description,transferred
0,0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [7]:
Holidays_Data.to_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\Holidays_Data.csv', index=False)
Holidays_Data = pd.read_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\Holidays_Data.csv')
Holidays_Data.head()

,Unnamed: 0,date,type,locale,locale_name,description,transferred
0,0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [8]:
oil_data = pd.read_csv('C:\\Users\\USER\\Downloads\\oil_data.csv')
oil_data.to_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\oil_data.csv', index=False)
oil_data = pd.read_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\oil_data.csv')
oil_data.head()

,Unnamed: 0,date,dcoilwtico
0,0,2013-01-01,NaN
1,1,2013-01-02,93.139999
2,2,2013-01-03,92.970001
3,3,2013-01-04,93.120003
4,4,2013-01-07,93.199997


In [9]:
stores_data = pd.read_csv('C:\\Users\\USER\\Downloads\\stores_data.csv')
stores_data.to_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\stores_data.csv', index=False)
stores_data = pd.read_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\stores_data.csv')
stores_data.head()

,Unnamed: 0,store_nbr,city,state,type,cluster
0,0,1,Quito,Pichincha,D,13
1,1,2,Quito,Pichincha,D,13
2,2,3,Quito,Pichincha,D,8
3,3,4,Quito,Pichincha,D,9
4,4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [10]:
test_data = pd.read_csv('C:\\Users\\USER\\Downloads\\test.csv')
test_data.to_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\test_data.csv', index=False)
test_data = pd.read_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\test_data.csv')
test_data.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [11]:
train_data = pd.read_csv('C:\\Users\\USER\\Downloads\\train_data.csv')
train_data.to_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\train_data.csv', index=False)
train_data = pd.read_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\train_data.csv')
train_data.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [12]:
Transactions_Data = pd.read_csv('C:\\Users\\USER\\Downloads\\transactions.csv')
Transactions_Data.to_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\Transactions_Data.csv', index=False)
Transactions_Data = pd.read_csv('C:\\Users\\USER\\.vscode\\Data_Science_Projects\\Data_Science_Projects\\Data\\Transactions_Data.csv')
Transactions_Data.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


## Data Cleaning and Exploration